In [1]:
import pandas as pd
import json
import string

In [2]:
all_files_ng = pd.read_csv("ClassificationNG/all_files_NGv1.csv")

In [3]:
def getanartistsjson(all_files_ng, artist_name):

    artist_df = all_files_ng[all_files_ng["main_artist"] == artist_name]
    
    num_of_streams = artist_df['streams'].sum()
    min_date = artist_df['End Date'].min()
    max_date = artist_df['End Date'].max()

    unique_songs = artist_df['track_name'].unique().tolist()
    num_unique_songs = len(unique_songs)
    
    source_end_date = artist_df[['End Date', "source"]].drop_duplicates()
    source_end_date_latest = pd.DataFrame(source_end_date.groupby("source")["End Date"].max())
    source_end_date_latest = source_end_date_latest.reset_index()
    
    rank_max_end_date = artist_df[['End Date', 'track_name', "rank"]].drop_duplicates()

    min_rank = rank_max_end_date["rank"].min()
    max_rank = rank_max_end_date["rank"].max()

    min_rank_song_dt = rank_max_end_date[rank_max_end_date["rank"] == min_rank]
    max_rank_song_dt = rank_max_end_date[rank_max_end_date["rank"] == max_rank]

    min_rank_song_unique_dt = min_rank_song_dt[["End Date", "track_name"]].drop_duplicates()
    max_rank_song_unique_dt = max_rank_song_dt[["End Date", "track_name"]].drop_duplicates()

    source_end_date_latest_ls = source_end_date_latest.values.tolist()

    min_rank_song_unique_dt_ls = min_rank_song_unique_dt.values.tolist()
    max_rank_song_unique_dt_ls = max_rank_song_unique_dt.values.tolist()
    
    artist_inner_json = {}
    
    artist_inner_json["num_of_streams"] = str(num_of_streams)
    artist_inner_json["min_rank"] = str(min_rank)
    artist_inner_json["max_rank"] = str(max_rank)
    artist_inner_json["min_date"] = min_date
    artist_inner_json["max_date"] = max_date
    artist_inner_json["num_unique_songs"] = str(num_unique_songs)

    artist_inner_json["unique_songs"] = unique_songs

    artist_inner_json["source_end_date_latest_ls"] = dict(source_end_date_latest_ls)
    artist_inner_json["min_rank_song_unique_dt_ls"] = dict(min_rank_song_unique_dt_ls)
    artist_inner_json["max_rank_song_unique_dt_ls"] = dict(max_rank_song_unique_dt_ls)
    
    artist_json = {}
    artist_json[artist_name] = artist_inner_json
    
    return artist_json

In [4]:
unique_artists = all_files_ng["main_artist"].unique().tolist()
    
for artist_name in unique_artists:  
    artist_json = getanartistsjson(all_files_ng, artist_name)
    
    # Serializing json
    json_object = json.dumps(artist_json, indent=4)
    
    new_artist_name = artist_name.translate(str.maketrans('', '', string.punctuation))

    # Writing to sample.json
    with open("ng_artists/" + new_artist_name +".json", "w") as outfile:
        outfile.write(json_object)